# Análsis Crediticio BankChurners

## Librerias

In [0]:
!pip install missingno

  Obtaining dependency information for missingno from https://files.pythonhosted.org/packages/87/22/cd5cf999af21c2f97486622c551ac3d07361ced8125121e907f588ff5f24/missingno-0.5.2-py3-none-any.whl.metadata
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
!pip install imblearn

  Obtaining dependency information for imblearn from https://files.pythonhosted.org/packages/81/a7/4179e6ebfd654bd0eac0b9c06125b8b4c96a9d0a8ff9e9507eb2a26d2d7e/imblearn-0.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for imbalanced-learn from https://files.pythonhosted.org/packages/d8/0d/c3bfccc5d460eec8ff56889802aa88f5d07280d5282b307a74558e6edc44/imbalanced_learn-0.12.4-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/258.3 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━ 204.8/258.3 kB 6.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 5.7 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import pandas as pd
import numpy as np

import seaborn as sns   
from matplotlib import pyplot as plt
import missingno as msno
import plotly.express as px


from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler
import multiprocessing
from sklearn.metrics import accuracy_score, precision_score, precision_score, recall_score, auc, f1_score, auc, roc_curve, confusion_matrix, classification_report

import mlflow
import mlflow.sklearn

from collections import Counter

import warnings
warnings.filterwarnings('ignore')

## Importación de información

In [0]:
url = 'https://raw.githubusercontent.com/spulidov91/DSA_churn/refs/heads/main/data/BankChurners.csv'
datos = pd.read_csv(url, sep = ',', on_bad_lines='skip' )
#Se eliminan las variables de estimación de naive Bayes
datos.drop(columns = ['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2' ], inplace = True)

## Preprocesamiento de datos

In [0]:
col_cat = datos.drop('Attrition_Flag', axis=1).select_dtypes(exclude=['int64','float64']).columns
col_num = datos.drop(['Attrition_Flag','CLIENTNUM'], axis=1).select_dtypes(include=['int64','float64']).columns

# Elimina una columna llamada 'CLIENTNUM'
datos = datos.drop('CLIENTNUM', axis=1)

# Se reemplaza la variable respuesta por 0 y 1
datos['Attrition_Flag'] = datos['Attrition_Flag'].replace({'Attrited Customer':1, 'Existing Customer':0})
# Se reemplaza en la columna gender F -> 1, M -> 0
datos['Gender'] = datos['Gender'].replace({'F':1,'M':0})

# Aplica one-hot encoding directamente sobre todas las columnas categóricas en categorical_columns
datos = pd.get_dummies(datos, columns=col_cat, drop_first=False, dtype=int)

# Crear una instancia de StandardScaler
scaler = StandardScaler()

# Aplicar StandardScaler solo a las columnas en col_num
datos[col_num] = scaler.fit_transform(datos[col_num])

# División de datos
y = datos.pop('Attrition_Flag')
X = datos

x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=69)

#Aplicando SMOTE a los datos de entrenamiento para tratar el desbalanceo de clases
sm = SMOTE(sampling_strategy='auto', random_state=1234)
x_smote, y_smote = sm.fit_resample(x_train, y_train)

#Realizando under/over sampling
over = BorderlineSMOTE(sampling_strategy=0.3)
under = RandomUnderSampler(sampling_strategy=0.6)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
x_smote_us, y_smote_us = pipeline.fit_resample(x_train, y_train)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f1042e6f880>
Traceback (most recent call last):
  File "/databricks/python/lib/python3.11/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/databricks/python/lib/python3.11/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.11/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
                   ^^^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.11/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
             ^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'split'
Ex

## Modelamiento

In [0]:
#Función para evaluación de métricas
def metrics(y_test , y_pred, y_proba):
  acc = accuracy_score(y_true=y_test,y_pred=y_pred)
  prec = precision_score(y_true=y_test,y_pred=y_pred)
  rec = recall_score(y_true=y_test,y_pred=y_pred)
  f1 = f1_score(y_true=y_test,y_pred=y_pred)

  fpr, tpr, thresholds = roc_curve(y_test,y_score =y_proba, pos_label=0)
  AUC_score =auc(fpr, tpr)
  return {'Accuracy':acc,
          'Precission' :prec,
          'Recall': rec,
          'F1 score': f1,
          'AUC': AUC_score}

In [0]:
#Función para correr experimentos Gradient Boosting
def experiment_run_gbc(exp, x_train, y_train, dataframe, learning_rate, max_iter, max_depth,x_test=x_test, y_test=y_test):
  #Definiendo el experimento
  experiment = mlflow.set_experiment(f'/Users/spulidov91@gmail.com/{exp}')

  with mlflow.start_run(experiment_id=experiment.experiment_id):
    #Definiendo parámetros
    learning_rate = learning_rate
    max_iter = max_iter
    max_depth = max_depth

    #Entrenando el modelo
    gb_sm_model = HistGradientBoostingClassifier(learning_rate = learning_rate, max_iter = max_iter, max_depth = max_depth)
    gb_sm_model.fit(x_train, y_train)

    #Generando predicciones
    y_sm_pred = gb_sm_model.predict(x_test)
    y_sm_pred_p = gb_sm_model.predict_proba(x_test)[:,0]

    #Generando métricas
    results = metrics(y_test,y_sm_pred,y_sm_pred_p)

    #Registros en ML Flow
    #Parámetros
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("max_iter", max_iter)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("Data_train", dataframe)

    #Modelo
    mlflow.sklearn.log_model(gb_sm_model, "Histogram-based Gradient Boosting Classification")
    #Métricas
    for metric in results.keys():
      mlflow.log_metric(metric, round(results[metric],5))

### Gradient Boosting

In [0]:
#Experimentos usando dataset desbalanceado
experiment_run_gbc(exp='bank-churn-GBC2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   learning_rate=0.005, max_iter=100, max_depth=5)
                   
experiment_run_gbc(exp='bank-churn-GBC2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   learning_rate=0.005, max_iter=200, max_depth=5)

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   learning_rate=0.005, max_iter=300, max_depth=5)       

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   learning_rate=0.005, max_iter=100, max_depth=10)
                   
experiment_run_gbc(exp='bank-churn-GBC2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   learning_rate=0.005, max_iter=200, max_depth=10)

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   learning_rate=0.005, max_iter=300, max_depth=10)     

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   learning_rate=0.005, max_iter=100, max_depth=5)
                   
experiment_run_gbc(exp='bank-churn-GBC2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   learning_rate=0.01, max_iter=200, max_depth=5)

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   learning_rate=0.01, max_iter=300, max_depth=5)       

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   learning_rate=0.01, max_iter=100, max_depth=10)
                   
experiment_run_gbc(exp='bank-churn-GBC2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   learning_rate=0.01, max_iter=200, max_depth=10)

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   learning_rate=0.01, max_iter=300, max_depth=10)    

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   learning_rate=0.02, max_iter=200, max_depth=5)

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   learning_rate=0.02, max_iter=300, max_depth=5)       

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   learning_rate=0.02, max_iter=100, max_depth=10)
                   
experiment_run_gbc(exp='bank-churn-GBC2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   learning_rate=0.02, max_iter=200, max_depth=10)

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   learning_rate=0.02, max_iter=300, max_depth=10)    

2024/11/09 21:10:18 INFO mlflow.tracking.fluent: Experiment with name '/Users/spulidov91@gmail.com/bank-churn-GBC2' does not exist. Creating a new experiment.
2024/11/09 21:10:23 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.4/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/11/09 21:10:30 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.4/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/11/09 21:10:38 WARNING mlflow.models.model: Model logged without a

In [0]:
#Experimentos usando SMOTE dataset
experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   learning_rate=0.005, max_iter=100, max_depth=5)
                   
experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   learning_rate=0.005, max_iter=200, max_depth=5)

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   learning_rate=0.005, max_iter=300, max_depth=5)       

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   learning_rate=0.005, max_iter=100, max_depth=10)
                   
experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   learning_rate=0.005, max_iter=200, max_depth=10)

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   learning_rate=0.005, max_iter=300, max_depth=10)     

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   learning_rate=0.005, max_iter=100, max_depth=5)
                   
experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   learning_rate=0.01, max_iter=200, max_depth=5)

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   learning_rate=0.01, max_iter=300, max_depth=5)       

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   learning_rate=0.01, max_iter=100, max_depth=10)
                   
experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   learning_rate=0.01, max_iter=200, max_depth=10)

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   learning_rate=0.01, max_iter=300, max_depth=10)    

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   learning_rate=0.02, max_iter=200, max_depth=5)

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   learning_rate=0.02, max_iter=300, max_depth=5)       

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   learning_rate=0.02, max_iter=100, max_depth=10)
                   
experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   learning_rate=0.02, max_iter=200, max_depth=10)

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   learning_rate=0.02, max_iter=300, max_depth=10)    

2024/11/09 21:12:28 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.4/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/11/09 21:12:35 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.4/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/11/09 21:12:43 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Ple

In [0]:
#Experimentos usando SMOTE over y under dataset
experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   learning_rate=0.005, max_iter=100, max_depth=5)
                   
experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   learning_rate=0.005, max_iter=200, max_depth=5)

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   learning_rate=0.005, max_iter=300, max_depth=5)       

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   learning_rate=0.005, max_iter=100, max_depth=10)
                   
experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   learning_rate=0.005, max_iter=200, max_depth=10)

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   learning_rate=0.005, max_iter=300, max_depth=10)     

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   learning_rate=0.005, max_iter=100, max_depth=5)
                   
experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   learning_rate=0.01, max_iter=200, max_depth=5)

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   learning_rate=0.01, max_iter=300, max_depth=5)       

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   learning_rate=0.01, max_iter=100, max_depth=10)
                   
experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   learning_rate=0.01, max_iter=200, max_depth=10)

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   learning_rate=0.01, max_iter=300, max_depth=10)    

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   learning_rate=0.02, max_iter=200, max_depth=5)

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   learning_rate=0.02, max_iter=300, max_depth=5)       

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   learning_rate=0.02, max_iter=100, max_depth=10)
                   
experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   learning_rate=0.02, max_iter=200, max_depth=10)

experiment_run_gbc(exp='bank-churn-GBC2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   learning_rate=0.02, max_iter=300, max_depth=10)    

2024/11/09 21:14:42 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.4/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/11/09 21:14:49 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.4/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/11/09 21:14:57 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Ple

### Random Forest

In [0]:
def experiment_run_rfc(exp, x_train, y_train, dataframe, n_estimators, criterion, max_depth,x_test=x_test, y_test=y_test):
  #Definiendo el experimento
  experiment = mlflow.set_experiment(f'/Users/spulidov91@gmail.com/{exp}')

  with mlflow.start_run(experiment_id=experiment.experiment_id):
    #Definiendo parámetros
    n_estimators = n_estimators
    criterion  = criterion
    max_depth = max_depth

    #Entrenando el modelo
    RF = RandomForestClassifier(n_estimators = n_estimators, criterion = criterion, max_depth = max_depth)
    RF.fit(x_train, y_train)

    #Generando predicciones
    y_sm_pred = RF.predict(x_test)
    y_sm_pred_p = RF.predict_proba(x_test)[:,0]

    #Generando métricas
    results = metrics(y_test,y_sm_pred,y_sm_pred_p)

    #Registros en ML Flow
    #Parámetros
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("criterion", criterion)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("Data_train", dataframe)
    #Modelo
    mlflow.sklearn.log_model(RF, "Random Forest Classifier")
    #Métricas
    for metric in results.keys():
      mlflow.log_metric(metric, round(results[metric],5))

In [0]:
#Experimentos con dataframe original
experiment_run_rfc(exp='bank-churn-RF2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   n_estimators=100, criterion='gini', max_depth=5)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   n_estimators=200, criterion='gini', max_depth=5)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   n_estimators=300, criterion='gini', max_depth=5)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   n_estimators=100, criterion='gini', max_depth=10)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   n_estimators=200, criterion='gini', max_depth=10)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   n_estimators=300, criterion='gini', max_depth=10)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   n_estimators=100, criterion='gini', max_depth=15)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   n_estimators=200, criterion='gini', max_depth=15)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   n_estimators=300, criterion='gini', max_depth=15)            
experiment_run_rfc(exp='bank-churn-RF2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   n_estimators=100, criterion='entropy', max_depth=5)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   n_estimators=200, criterion='entropy', max_depth=5)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   n_estimators=300, criterion='entropy', max_depth=5)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   n_estimators=100, criterion='entropy', max_depth=10)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   n_estimators=200, criterion='entropy', max_depth=10)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   n_estimators=300, criterion='entropy', max_depth=10)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   n_estimators=100, criterion='entropy', max_depth=15)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   n_estimators=200, criterion='entropy', max_depth=15)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='Unbalanced',
                   x_train = x_train,y_train= y_train,   
                   n_estimators=300, criterion='entropy', max_depth=15)            
                 

2024/11/09 21:16:39 INFO mlflow.tracking.fluent: Experiment with name '/Users/spulidov91@gmail.com/bank-churn-RF2' does not exist. Creating a new experiment.
2024/11/09 21:16:44 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.4/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/11/09 21:16:51 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.4/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/11/09 21:17:00 WARNING mlflow.models.model: Model logged without a 

In [0]:

#Experimentos con dataframe SMOTE
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   n_estimators=100, criterion='gini', max_depth=5)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   n_estimators=200, criterion='gini', max_depth=5)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   n_estimators=300, criterion='gini', max_depth=5)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   n_estimators=100, criterion='gini', max_depth=10)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   n_estimators=200, criterion='gini', max_depth=10)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   n_estimators=300, criterion='gini', max_depth=10)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   n_estimators=100, criterion='gini', max_depth=15)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   n_estimators=200, criterion='gini', max_depth=15)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   n_estimators=300, criterion='gini', max_depth=15)            

experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   n_estimators=100, criterion='entropy', max_depth=5)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   n_estimators=200, criterion='entropy', max_depth=5)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   n_estimators=300, criterion='entropy', max_depth=5)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   n_estimators=100, criterion='entropy', max_depth=10)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   n_estimators=200, criterion='entropy', max_depth=10)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   n_estimators=300, criterion='entropy', max_depth=10)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   n_estimators=100, criterion='entropy', max_depth=15)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   n_estimators=200, criterion='entropy', max_depth=15)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE',
                   x_train = x_smote,y_train= y_smote,   
                   n_estimators=300, criterion='entropy', max_depth=15)            
                 

2024/11/09 21:19:20 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.4/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/11/09 21:19:28 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.4/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/11/09 21:19:38 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Ple

In [0]:
#Experimentos con dataframe SMOTE Under/Over
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   n_estimators=100, criterion='gini', max_depth=5)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   n_estimators=200, criterion='gini', max_depth=5)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   n_estimators=300, criterion='gini', max_depth=5)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   n_estimators=100, criterion='gini', max_depth=10)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   n_estimators=200, criterion='gini', max_depth=10)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   n_estimators=300, criterion='gini', max_depth=10)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   n_estimators=100, criterion='gini', max_depth=15)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   n_estimators=200, criterion='gini', max_depth=15)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   n_estimators=300, criterion='gini', max_depth=15)            

experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   n_estimators=100, criterion='entropy', max_depth=5)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   n_estimators=200, criterion='entropy', max_depth=5)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   n_estimators=300, criterion='entropy', max_depth=5)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   n_estimators=100, criterion='entropy', max_depth=10)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   n_estimators=200, criterion='entropy', max_depth=10)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   n_estimators=300, criterion='entropy', max_depth=10)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   n_estimators=100, criterion='entropy', max_depth=15)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   n_estimators=200, criterion='entropy', max_depth=15)
experiment_run_rfc(exp='bank-churn-RF2',dataframe='SMOTE-Over/Under',
                   x_train = x_smote_us,y_train= y_smote_us,   
                   n_estimators=300, criterion='entropy', max_depth=15)            
                 

2024/11/09 21:22:39 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.4/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/11/09 21:22:46 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.4/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/11/09 21:22:53 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Ple